In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator

from indra_db_lite.api import get_entrez_pmids_for_hgnc
from indra_db_lite.api import get_entrez_pmids_for_uniprot
from indra_db_lite.api import get_plaintexts_for_text_ref_ids
from indra_db_lite.api import get_text_ref_ids_for_agent_text
from indra_db_lite.api import get_text_ref_ids_for_pmids


from adeft_indra.grounding import AdeftGrounder
from adeft_indra.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename

In [2]:
def get_text_ref_ids_for_entity(ns, id_):
    if ns == 'HGNC':
        pmids = get_entrez_pmids_for_hgnc(id_)
    elif ns == 'UP':
        pmids = get_entrez_pmids_for_uniprot(id_)
    return list(get_text_ref_ids_for_pmids(pmids).values())

In [3]:
adeft_grounder = AdeftGrounder()

In [4]:
shortforms = ['NRP', 'NRPs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../../', 'results', model_name))

In [5]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    text_ref_ids = get_text_ref_ids_for_agent_text(shortform)
    content = get_plaintexts_for_text_ref_ids(text_ref_ids, contains=shortforms)
    text_dict = content.flatten()
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [6]:
list(zip(longforms, counts))

[('neuropilin', 19),
 ('nonribosomal peptides', 16),
 ('non ribosomal peptides', 14),
 ('no reflow phenomenon', 10),
 ('normothermic regional perfusion', 9),
 ('nucleolin related protein', 8),
 ('non ribosomal peptide', 7),
 ('nephrotic range proteinuria', 6),
 ('n rich protein', 6),
 ('neonatal resuscitation program', 5),
 ('nucleus raphe pallidus', 5),
 ('nemo related protein', 4),
 ('nonreplicating persistence', 4),
 ('non rephasing', 3),
 ('nonribosomal peptide', 3),
 ('nitroxide radical containing polymer', 3),
 ('non ribosomal synthesized peptides', 3)]

In [7]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

In [8]:
result = [grounding_map, names, pos_labels]

In [9]:
result

[{'n rich protein': 'UP:Q8RXN8',
  'nemo related protein': 'HGNC:17142',
  'neonatal resuscitation program': 'ungrounded',
  'nephrotic range proteinuria': 'HP:HP:0012593',
  'neuropilin': 'MESH:D039921',
  'nitroxide radical containing polymer': 'ungrounded',
  'no reflow phenomenon': 'MESH:D054318',
  'non rephasing': 'ungrounded',
  'non ribosomal peptide': 'nonribosomal_peptides',
  'non ribosomal peptides': 'nonribosomal_peptides',
  'non ribosomal synthesized peptides': 'ungrounded',
  'nonreplicating persistence': 'GO:GO:0085015',
  'nonribosomal peptide': 'nonribosomal_peptides',
  'nonribosomal peptides': 'nonribosomal_peptides',
  'normothermic regional perfusion': 'ungrounded',
  'nucleolin related protein': 'nucleolin_related_protein',
  'nucleus raphe pallidus': 'MESH:D065848'},
 {'UP:Q8RXN8': 'NRP',
  'HGNC:17142': 'OPTN',
  'HP:HP:0012593': 'Nephrotic range proteinuria',
  'MESH:D039921': 'Neuropilins',
  'MESH:D054318': 'No-Reflow Phenomenon',
  'nonribosomal_peptides':

In [12]:
grounding_map, names, pos_labels = [
    {'n rich protein': 'UP:Q8RXN8',
  'nemo related protein': 'HGNC:17142',
  'neonatal resuscitation program': 'ungrounded',
  'nephrotic range proteinuria': 'HP:HP:0012593',
  'neuropilin': 'MESH:D039921',
  'nitroxide radical containing polymer': 'ungrounded',
  'no reflow phenomenon': 'MESH:D054318',
  'non rephasing': 'ungrounded',
  'non ribosomal peptide': 'nonribosomal_peptides',
  'non ribosomal peptides': 'nonribosomal_peptides',
  'non ribosomal synthesized peptides': 'nonribosomal_peptides',
  'nonreplicating persistence': 'GO:GO:0085015',
  'nonribosomal peptide': 'nonribosomal_peptides',
  'nonribosomal peptides': 'nonribosomal_peptides',
  'normothermic regional perfusion': 'ungrounded',
  'nucleolin related protein': 'nucleolin_related_protein',
  'nucleus raphe pallidus': 'MESH:D065848'},
 {'UP:Q8RXN8': 'NRP',
  'HGNC:17142': 'OPTN',
  'HP:HP:0012593': 'Nephrotic range proteinuria',
  'MESH:D039921': 'Neuropilins',
  'MESH:D054318': 'No-Reflow Phenomenon',
  'nonribosomal_peptides': 'nonribosomal_peptides',
  'GO:GO:0085015': 'dormancy maintenance of symbiont in host',
  'nucleolin_related_protein': 'nucleolin_related_protein',
  'MESH:D065848': 'Nucleus Raphe Pallidus'},
 ['MESH:D039921']
]


In [13]:
excluded_longforms = []

In [14]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [16]:
additional_entities = {
    'HGNC:17142': ['OPTN', ['NRP', 'optineurin', 'OPTN']],
    'HGNC:7637': ['NAP1L1', ['NRP', 'NAP1L1', 'NAP1', 'NAP1L']],
    'UP:Q8RXN8': ['NRP', ['NRP']],
}

In [17]:
unambiguous_agent_texts = {}

In [18]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts(
    (text, text_ref_id) for text_ref_id, text in all_texts.items()
)
agent_text_text_ref_id_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_text_ref_id_map[label].append(id_)

entity_text_ref_id_map = {
    entity: set(
        get_text_ref_ids_for_entity(*entity.split(':', maxsplit=1))
    )
    for entity in additional_entities
}

In [19]:
intersection1 = []
for entity1, trids1 in entity_text_ref_id_map.items():
    for entity2, trids2 in entity_text_ref_id_map.items():
        intersection1.append((entity1, entity2, len(trids1 & trids2)))

In [20]:
intersection2 = []
for entity1, trids1 in agent_text_text_ref_id_map.items():
    for entity2, pmids2 in entity_text_ref_id_map.items():
        intersection2.append((entity1, entity2, len(set(trids1) & trids2)))

In [21]:
intersection1

[('HGNC:17142', 'HGNC:17142', 249),
 ('HGNC:17142', 'HGNC:7637', 15),
 ('HGNC:17142', 'UP:Q8RXN8', 0),
 ('HGNC:7637', 'HGNC:17142', 15),
 ('HGNC:7637', 'HGNC:7637', 177),
 ('HGNC:7637', 'UP:Q8RXN8', 0),
 ('UP:Q8RXN8', 'HGNC:17142', 0),
 ('UP:Q8RXN8', 'HGNC:7637', 0),
 ('UP:Q8RXN8', 'UP:Q8RXN8', 7)]

In [22]:
intersection2

[('HGNC:17142', 'HGNC:17142', 0),
 ('HGNC:17142', 'HGNC:7637', 0),
 ('HGNC:17142', 'UP:Q8RXN8', 0),
 ('nonribosomal_peptides', 'HGNC:17142', 0),
 ('nonribosomal_peptides', 'HGNC:7637', 0),
 ('nonribosomal_peptides', 'UP:Q8RXN8', 0),
 ('UP:Q8RXN8', 'HGNC:17142', 0),
 ('UP:Q8RXN8', 'HGNC:7637', 0),
 ('UP:Q8RXN8', 'UP:Q8RXN8', 0),
 ('MESH:D065848', 'HGNC:17142', 0),
 ('MESH:D065848', 'HGNC:7637', 0),
 ('MESH:D065848', 'UP:Q8RXN8', 0),
 ('MESH:D039921', 'HGNC:17142', 0),
 ('MESH:D039921', 'HGNC:7637', 0),
 ('MESH:D039921', 'UP:Q8RXN8', 0),
 ('ungrounded', 'HGNC:17142', 0),
 ('ungrounded', 'HGNC:7637', 0),
 ('ungrounded', 'UP:Q8RXN8', 0),
 ('nucleolin_related_protein', 'HGNC:17142', 0),
 ('nucleolin_related_protein', 'HGNC:7637', 0),
 ('nucleolin_related_protein', 'UP:Q8RXN8', 0),
 ('MESH:D054318', 'HGNC:17142', 0),
 ('MESH:D054318', 'HGNC:7637', 0),
 ('MESH:D054318', 'UP:Q8RXN8', 0),
 ('GO:GO:0085015', 'HGNC:17142', 0),
 ('GO:GO:0085015', 'HGNC:7637', 0),
 ('GO:GO:0085015', 'UP:Q8RXN8', 0)

In [23]:
all_used_trids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_trids = set()
    for agent_text in agent_texts[1]:
        trids = set(get_text_ref_ids_for_agent_text(agent_text))
        new_trids = list(trids - all_texts.keys() - used_trids)
        content = get_plaintexts_for_text_ref_ids(new_trids, contains=agent_texts[1])
        text_dict = content.flatten()
        corpus.extend(
            [
                (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
            ]
        )
        used_trids.update(new_trids)
    all_used_trids.update(used_trids)
        
for entity, trids in entity_text_ref_id_map.items():
    new_trids = list(set(trids) - all_texts.keys() - all_used_trids)
    _, contains = additional_entities[entity]
    content = get_plaintexts_for_text_ref_ids(new_trids, contains=contains)
    text_dict = content.flatten()
    corpus.extend(
        [
            (text, entity, trid) for trid, text in text_dict.items() if len(text) >= 5
        ]
    )

In [24]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [25]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2021-10-08 17:12:48] /adeft/Py/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2021-10-08 17:12:55] /adeft/Py/adeft/adeft/modeling/classify.py - Best f1 score of 0.976084920769895 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [26]:
classifier.stats

{'label_distribution': {'HGNC:17142': 175,
  'nonribosomal_peptides': 39,
  'UP:Q8RXN8': 6,
  'MESH:D065848': 4,
  'MESH:D039921': 23,
  'ungrounded': 14,
  'nucleolin_related_protein': 2,
  'MESH:D054318': 3,
  'GO:GO:0085015': 3,
  'HP:HP:0012593': 2,
  'HGNC:7637': 20},
 'f1': {'mean': 0.976085, 'std': 0.016823},
 'precision': {'mean': 0.962508, 'std': 0.03427},
 'recall': {'mean': 0.990909, 'std': 0.011134},
 'GO:GO:0085015': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'HGNC:17142': {'f1': {'mean': 0.975477, 'std': 0.02283},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.953078, 'std': 0.042619}},
 'HGNC:7637': {'f1': {'mean': 0.942857, 'std': 0.069985},
  'pr': {'mean': 0.9, 'std': 0.122474},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'HP:HP:0012593': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D039921': {'f1': {'mean': 1.0, 'std': 0.0},

In [27]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [28]:
disamb.dump(model_name, results_path)

In [29]:
print(disamb.info())

Disambiguation model for NRP, and NRPs

Produces the disambiguations:
	NAP1L1*	HGNC:7637
	NRP*	UP:Q8RXN8
	Nephrotic range proteinuria	HP:HP:0012593
	Neuropilins*	MESH:D039921
	No-Reflow Phenomenon	MESH:D054318
	Nucleus Raphe Pallidus	MESH:D065848
	OPTN*	HGNC:17142
	dormancy maintenance of symbiont in host	GO:GO:0085015
	nonribosomal_peptides	nonribosomal_peptides
	nucleolin_related_protein	nucleolin_related_protein

Class level metrics:
--------------------
Grounding                               	Count	F1     
                                    OPTN*	175	0.97548
                   nonribosomal_peptides	 39	0.95137
                             Neuropilins*	 23	    1.0
                                  NAP1L1*	 20	0.94286
                              Ungrounded	 14	0.83143
                                     NRP*	  6	    1.0
                  Nucleus Raphe Pallidus	  4	    0.6
                    No-Reflow Phenomenon	  3	    0.4
dormancy maintenance of symbiont in host	  3	    0.4
  

In [30]:
model_to_s3(disamb)

In [26]:
from adeft.disambiguate import load_disambiguator

In [27]:
disamb = load_disambiguator("BAL")

In [28]:
disamb

In [29]:
print(_28.info())

Disambiguation model for BAL

Produces the disambiguations:
	Bronchoalveolar Lavage	MESH:D018893
	CEL*	HGNC:1848
	Liver, Artificial	MESH:D019164
	benzaldehyde lyase*	MESH:C059416
	betaine aldehyde*	CHEBI:CHEBI:15710
	dimercaprol*	CHEBI:CHEBI:64198

Class level metrics:
--------------------
Grounding             	Count	F1     
Bronchoalveolar Lavage	1259	 0.9929
                   CEL*	  36	    1.0
     Liver, Artificial	  18	0.83619
            Ungrounded	  17	   0.65
           dimercaprol*	   8	    0.4
    benzaldehyde lyase*	   3	    0.2
      betaine aldehyde*	   2	    0.2

Global Metrics:
-----------------
	F1 score:	0.90773
	Precision:	1.0
	Recall:		0.83293

* Positive labels
See Docstring for explanation



/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/adeft/.virtualenvs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is 